In [56]:
import numpy as np
from cvxopt import matrix,solvers
import math 
import time

We have done the production planning application and defined the A,B and C matrix in standard form(max C^T.x such 
that Ax<=b, modify those accordingly for different examples,(for max C^T.x with constraint AX <=B, C array values should be written in negative, if we have to maximize function )

In [57]:

A=np.array([[100,60,120,70],[-2,-1,0,0],[0,0,-3,-2],[1,1,0,0],[0,0,1,1]]).astype(np.float64)
B=np.array([990,-7,-16,10,6]).astype(np.float64)
C=np.array([-40,-20,-30,-10]).astype(np.float64)
solution=[]
iterations=[]

In Branch and Cut Algorithm, we use both branching and cuts, so different preprocessing is required for both the algorithms, which is what we have done below 

In [58]:
def preprocessing_CP(c,a,b):
    Nc=np.shape(a)[0] #  no of constraints
    c=np.hstack((c,(np.zeros(Nc+1))))
    I=np.eye(Nc)
    tableau=np.append(a,I,axis=1)
    tableau=np.vstack([tableau.T,b])
    tableau=tableau.T
    tableau=np.vstack([tableau,c])
    return tableau

In [59]:
def preprocessing_BAB(c,a,b) :
    Nc=np.shape(a)[0]
    Nv=np.shape(a)[1]
    b=np.hstack((b,(np.zeros(Nv))))
    I=np.eye(Nv)
    a=np.append(a,-I,axis=0)
    c=matrix(c)
    b=matrix(b)
    a=matrix(a)
    return c,a,b
    

We have written the code for solving simplex tableau through which we will generate the cuts

In [60]:
def simplex(a,A):
    fv=[]
    infeasible=0
    Nc=np.shape(A)[0]
    Nv=np.shape(A)[1]
    for i in range(Nc):
        fv.append(Nv+i)
    for k in range(1001):
        if(np.all(a[-1,0:-1]>=0) and np.all(a[0:-1,-1]>=0)):
            return a,fv,infeasible
        else :
            column=np.argmin(a[-1,0:-1]) 
            if a[-1][column]<0:
                b=(a[0:-1,column])
                b=np.where(b==0,0.0000000001,b)
                ratio=((a[0:-1,-1])/b)
                ratio=np.where(ratio<0.0001,100000,ratio)
                for pivot in range(len(ratio)):
                    if ratio[pivot]==0 and b[pivot]<=0 :
                        ratio[pivot]=10000
                row=np.argmin(ratio)
                a[row,:]=a[row,:]/a[row][column]
                fv[row]=column
                for i in range(Nc+1):
                    if(i!=row):
                        a[i,:]=a[i,:]-a[i][column]*a[row,:]
                continue
            else :
                for i in range(Nc):  
                    if(a[i][-1]<-0.000001):
                        column=np.argmin(a[i,:-1])
                        if a[i][column]<0:
                            row=i
                            b=(a[0:-1,column])
                            b=np.where(b==0,0.0000000001,b)
                            ratio=((a[0:-1,-1])/b)
                            ratio=np.where(ratio<0.01,10000,ratio)
                            for pivot in range(len(ratio)):
                                if ratio[pivot]>=0 and ratio[pivot]<row and b[pivot]>0 :
                                    row=pivot
                            a[row,:]=a[row,:]/a[row][column]
                            fv[row]=column
                            for j in range(Nc+1):
                                if(j!=row):
                                    a[j,:]=a[j,:]-a[j][column]*a[row,:]
                            i=0
                        else :
                            infeasible=1
                            return a,fv,infeasible
    return a,fv,infeasible
            

The function below is used to generate one cut which we will add in our constraint, new constraint or cut generated 
will be shown in the output

In [61]:
def onecut(A,B,C):
    Nv=np.shape(A)[1] # no of variables
    flag=0
    Nc=np.shape(A)[0]
    D=np.zeros([Nv,Nv+1])
    for i in range(Nv):
        D[i,i]=-1
    E=np.vstack((A.T,B))
    E=E.T
    D=np.vstack((D,E,np.zeros(Nv+1)))
    D[-1,-1] = -1  # Size of D = Nc+1
    tableau=preprocessing_CP(C,A,B)
    sol_tab,solutions,infeasible=simplex(tableau,A)
    z=sol_tab-np.floor(sol_tab)
    x=np.zeros(Nv)
    for j in range(Nv) :
        for i in range(len(solutions)):
            if solutions[i]==j:
                x[j]=sol_tab[i,-1]
                
    new_const=np.zeros(Nv+1)
    
    if(infeasible==1) or np.any(sol_tab[-1,0:-1]<0) or np.any(sol_tab[0:-1,-1]<0):
        flag=100
        return A,B,x,sol_tab[-1][-1],flag,new_const
        
    if np.all(z[0:-1,-1]<=0.001) :
        flag=1
    
    else :
        k=0
        for i in range(np.shape(z)[0]-1):
            if(z[i,-1]>=0.001):
                k=i
                for j in range(0,Nv+Nc+1):
                    new_const+=z[k][j]*D[j,:]  
                new_const=np.round(new_const,3)  
                A=np.vstack([A,new_const[:-1]])
                B=np.hstack((B,new_const[-1]))
                dupli=np.vstack((A.T,B))
                dupli=dupli.T
                dupli2=(np.unique(dupli,axis=0))
                if(np.shape(dupli)[0]==np.shape(dupli2)[0]):
                    break
                else :
                    A=A[:-1,:]
                    B=B[:-1]
        if(i==np.shape(z)[0]-2):
            flag=100
            return A,B,x,sol_tab[-1][-1],flag,new_const
            
        
    return A,B,x,sol_tab[-1][-1],flag,new_const

In [62]:
def w(sol):
    weigh=np.abs(C)
    for a in range(len(C)):
        if math.floor(sol[a])==math.ceil(sol[a]):
            weigh[a]=0
    return weigh

We have written the code for performing Branch and Cut below

In [63]:
def BAC(A,B,C):
    
    Nv=np.shape(A)[1] # no of variables
    for k in range(1):
        A,B,x,obj,flag,constraint=onecut(A,B,C)
        if flag==1:
            print("Solution complete ",x,obj)
            solution.append(x)
            return x
        elif flag==0:
            iterations.append(1)
            print("Solution incomplete ",x,obj,"\n Constraint : ",constraint)
        else:
            print("Constraint not found or solution is infeasible ")
    
    
    C0,A0,B0=preprocessing_BAB(C,A,B)
    iterations.append(1)
    sol=solvers.lp(C0,A0,B0)
    
    
    if sol['x'] is None:
        return 0
    sol['x']=np.round(sol['x'],3)
    print(sol['x'])
    
    weight=w(sol['x'])
    i=int(np.argmax(weight))
    d=[math.floor(sol['x'][i]),math.ceil(sol['x'][i])]
    
    if d[0]==d[1]:
        if(np.sum(weight)==0):
            solution.append(sol['x'])
        return sol['x']
    
    
        
    # ceil 
    z=np.zeros(len(C))
    z[i]=-1
    A1=np.vstack([A,z])
    B1=np.append(B,-d[1])
    sol1=BAC(A1,B1,C)
    # checkint(sol1)

    #floor
    z=np.zeros(len(C))
    z[i]=1
    A2=np.vstack([A,z])
    B2=np.append(B,d[0])
    sol2=BAC(A2,B2,C)
    #checkint(sol2)
    
    

Running the Branch and Cut algorithm

In [64]:
start_time=time.time()
BAC(A,B,C)          

Solution incomplete  [3.5        0.         5.33333333 0.        ] 300.0 
 Constraint :  [ 1.  0.  0. -1.  3.]
     pcost       dcost       gap    pres   dres   k/t
 0: -2.9110e+02 -2.8768e+03  1e+02  8e-03  8e+00  1e+00
 1: -2.9602e+02 -5.2355e+02  7e+00  7e-04  7e-01  6e-01
 2: -2.9659e+02 -3.2586e+02  8e-01  1e-04  1e-01  8e-02
 3: -2.9666e+02 -2.9801e+02  3e-02  4e-06  4e-03  3e-03
 4: -2.9667e+02 -2.9668e+02  3e-04  4e-08  4e-05  3e-05
 5: -2.9667e+02 -2.9667e+02  3e-06  4e-10  4e-07  3e-07
 6: -2.9667e+02 -2.9667e+02  3e-08  4e-12  4e-09  3e-09
Optimal solution found.
[[3.367]
 [0.283]
 [5.089]
 [0.367]]
Constraint not found or solution is infeasible 
     pcost       dcost       gap    pres   dres   k/t
 0: -2.9245e+02 -2.7929e+03  1e+02  9e-03  8e+00  1e+00
 1: -2.9561e+02 -6.7229e+02  1e+01  1e-03  1e+00  6e-01
 2: -2.8668e+02 -1.1760e+03  1e+02  3e-03  3e+00  4e+00
 3: -2.8014e+02 -6.6834e+02  8e+01  1e-03  1e+00  3e+00
 4: -2.9565e+02 -6.2584e+02  2e+02  1e-03  1e+00  1e+01


In [65]:
objective=np.zeros(len(solution))
for j in range(len(solution)):
    x=solution[j]
    objective[j]=np.matmul(C.T,x)
    #print(x[-n:],objective[j])
    
optimal=np.argmin(objective)
x=solution[optimal]
print(x,objective[optimal])

[[3.]
 [1.]
 [4.]
 [2.]] -280.0


In [66]:
print("No. of iterations : ",len(iterations))
print("Time Taken : ",np.round(time.time()-start_time,3))

No. of iterations :  5
Time Taken :  0.166
